In [1]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [2]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [5]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [6]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [7]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [11]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['Title'])
    data["Title"] = data["Title"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["Title"] = data['Title'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["Title"] = data["Title"].apply(remove_punctuations)
    data["Title"] = data['Title'].str.replace(r'\d+', '', regex=True)  # fixed
    data["Title"] = data["Title"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["Title"]

In [13]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [15]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    print(prediction)

In [31]:
txt = "කිසිවෙකු නොසිතූ ලෙස ශ්රී ලංකා කණ්ඩායමට පළමු ඉනිමේ ඇරඹුමේ අත් වූ ඉරණම"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)

prediction
print("result")

['Sport']
result
